In [1]:
import sys

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

In [2]:
# Test Pareto Optimizer

from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer
from robyn.tutorials.utils.data_mapper import import_output_models, import_input_collect, load_data_from_json

2024-11-14 18:34:10,109 - robyn - INFO - Logging is set up to console only.
/Users/yijuilee/robynpy_release_reviews/robynvenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data from JSON
inputCollect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_InputCollect.json"
)
outputModel = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_OutputModels.json"
)
input_collect = import_input_collect(inputCollect)
output_models = import_output_models(outputModel)

Rolling Window Start Index: 6
Rolling Window End Index: 162
Debug: R output data keys: ['trial1', 'trial2', 'trial3', 'trial4', 'trial5']
Data for trial1: Keys = ['resultCollect', 'hyperBoundNG', 'hyperBoundFixed']
Debug: resultCollect keys for trial1: ['resultHypParam', 'xDecompAgg', 'liftCalibration', 'decompSpendDist', 'iter', 'elapsed.min']
Debug: Sample resultHypParam for trial1: [{'facebook_S_alphas': 1.98202923325, 'facebook_S_gammas': 0.58424338159, 'facebook_S_thetas': 0.09072730547999999, 'newsletter_alphas': 1.7514167265, 'newsletter_gammas': 0.6585122811199999, 'newsletter_thetas': 0.22826231587, 'ooh_S_alphas': 1.9350874957500002, 'ooh_S_gammas': 0.80499137605, 'ooh_S_thetas': 0.26847073588000003, 'print_S_alphas': 1.3471347205000002, 'print_S_gammas': 0.6602001640299999, 'print_S_thetas': 0.34398200833000003, 'search_S_alphas': 1.835435262, 'search_S_gammas': 0.6915214566899999, 'search_S_thetas': 0.11253588111, 'tv_S_alphas': 1.71477961025, 'tv_S_gammas': 0.6107644945299

In [4]:
mmm_data = input_collect["mmm_data"]
# display(mmm_data.data.head())
# Display Model Outputs

output_models = output_models
# display((model_outputs.trials[0].result_hyp_param))

hyperparameters = input_collect["hyperparameters"]
# display(hyperparameters)

featurized_mmm_data = input_collect["featurized_mmm_data"]

holidays_data = input_collect["holidays_data"]

In [5]:
# Add after model training
print("Model Output Summary:")
print(f"Number of trials: {len(output_models.trials)}")
print(f"Average models per trial: {len(output_models.all_result_hyp_param) / len(output_models.trials)}")
print(f"Total unique models: {len(output_models.all_result_hyp_param['sol_id'].unique())}")

print("\nMetrics Distribution:")
metrics_df = output_models.all_result_hyp_param[["nrmse", "decomp.rssd", "mape"]]
print(metrics_df.describe())

# Additional validation to debug model output
print("\nColumns in result_hyp_param:")
print(output_models.all_result_hyp_param.columns.tolist())

print("\nSample rows of metrics:")
print(output_models.all_result_hyp_param[["sol_id", "nrmse", "decomp.rssd", "mape"]].head())

# Show shape of result dataframes
print("\nDataFrame Shapes:")
print(f"result_hyp_param: {output_models.all_result_hyp_param.shape}")
print(f"x_decomp_agg: {output_models.all_x_decomp_agg.shape}")
print(f"decomp_spend_dist: {output_models.all_decomp_spend_dist.shape}")

Model Output Summary:
Number of trials: 5
Average models per trial: 2007.0
Total unique models: 10035

Metrics Distribution:
              nrmse   decomp.rssd          mape
count  10035.000000  10035.000000  10035.000000
mean       0.285779      0.628863    176.847461
std        0.067437      0.083541    167.500223
min        0.085069      0.369959      0.812119
25%        0.242605      0.565598     87.377313
50%        0.264548      0.631424    115.026803
75%        0.301364      0.696620    182.792278
max        0.521505      0.873183   1129.896150

Columns in result_hyp_param:
['facebook_S_alphas', 'facebook_S_gammas', 'facebook_S_thetas', 'newsletter_alphas', 'newsletter_gammas', 'newsletter_thetas', 'ooh_S_alphas', 'ooh_S_gammas', 'ooh_S_thetas', 'print_S_alphas', 'print_S_gammas', 'print_S_thetas', 'search_S_alphas', 'search_S_gammas', 'search_S_thetas', 'tv_S_alphas', 'tv_S_gammas', 'tv_S_thetas', 'train_size', 'rsq_train', 'rsq_val', 'rsq_test', 'nrmse_train', 'nrmse_val', 'nrm

In [6]:
# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(mmm_data, output_models, hyperparameters, featurized_mmm_data, holidays_data)

In [7]:
# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=100)

2024-11-14 18:34:14 [INFO] Starting Pareto optimization
2024-11-14 18:34:14 [INFO] Starting model data aggregation
2024-11-14 18:34:14 [INFO] Computing Pareto fronts
2024-11-14 18:34:15 [INFO] Pareto front computation completed
2024-11-14 18:34:15 [INFO] Preparing Pareto data
2024-11-14 18:34:15 [INFO] Number of Pareto-optimal solutions found: 637
2024-11-14 18:34:15 [INFO] Selected 10 Pareto-fronts containing 102 candidates
2024-11-14 18:34:15 [INFO] Filtering data for selected Pareto fronts...
2024-11-14 18:34:15 [INFO] Calculating response curves for 510 models' media variables...
Processing rows: 100%|██████████| 510/510 [00:04<00:00, 117.49it/s] 
2024-11-14 18:34:22 [INFO] Successfully processed 510 response curves
2024-11-14 18:34:22 [INFO] Computing final metrics...
2024-11-14 18:34:22 [INFO] Calculating ROI and CPA metrics...
2024-11-14 18:34:22 [INFO] Starting plot data generation...
2024-11-14 18:34:22 [INFO] Processing Pareto front 1
2024-11-14 18:34:22 [INFO] Pareto-Front: 

In [8]:
pareto_result.plot_data_collect

{'2_197_8': {'plot1data': {'plotMediaShareLoopBar':            rn     nrmse  decomp.rssd  rsq_train      variable     value
   0        tv_S  0.108185     0.452288   0.938282   spend_share  0.206047
   1       ooh_S  0.108185     0.452288   0.938282   spend_share  0.655151
   2     print_S  0.108185     0.452288   0.938282   spend_share  0.047837
   3  facebook_S  0.108185     0.452288   0.938282   spend_share  0.026563
   4    search_S  0.108185     0.452288   0.938282   spend_share  0.064401
   5        tv_S  0.108185     0.452288   0.938282  effect_share  0.257470
   6       ooh_S  0.108185     0.452288   0.938282  effect_share  0.353539
   7     print_S  0.108185     0.452288   0.938282  effect_share  0.117501
   8  facebook_S  0.108185     0.452288   0.938282  effect_share  0.000000
   9    search_S  0.108185     0.452288   0.938282  effect_share  0.271490,
   'plotMediaShareLoopLine':             rn     nrmse  decomp.rssd  rsq_train   variable     value
   10        tv_S  0.10818

In [9]:
# 5. Check results
print("Pareto Optimization Results:")
print(f"Number of Pareto fronts: {pareto_result.pareto_solutions}")
print(f"MediaVecCollect: {pareto_result.media_vec_collect.shape, pareto_result.media_vec_collect}")
print("\Hyper parameter solutions:")
print(pareto_result.result_hyp_param)

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg)
print("\result Calibration:")
print(pareto_result.result_calibration)
print("\nx Decomp Vec Collect:")
print(pareto_result.x_decomp_vec_collect.shape, pareto_result.x_decomp_vec_collect)
print("\nCarryover percentage all:")
print(pareto_result.df_caov_pct_all.shape, pareto_result.df_caov_pct_all)
print("\Plot Data Collected")
# print("NUMBER OF PLOTS Data collected for:", len(pareto_result.plot_data_collect["2_4_8"]))
# print("Plot data for solid 3_206_6", pareto_result.plot_data_collect["2_4_8"])

# 6. Validate logic
assert pareto_result.pareto_fronts == "auto" or isinstance(
    pareto_result.pareto_fronts, int
), "Invalid pareto_fronts value"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")

Pareto Optimization Results:
Number of Pareto fronts: {'4_108_8', '2_208_9', '3_78_9', '3_95_4', '4_105_5', '2_97_3', '2_95_3', '2_215_9', '2_207_8', '3_91_7', '3_221_6', '3_201_2', '1_217_2', '3_147_7', '3_214_7', '3_204_9', '3_138_7', '3_98_7', '3_78_7', '3_158_7', '5_196_6', '2_198_9', '2_208_2', '1_166_9', '3_104_4', '3_108_7', '3_74_4', '1_150_3', '3_219_3', '3_200_1', '3_186_4', '3_114_4', '5_80_5', '3_131_1', '5_109_5', '3_91_1', '3_71_1', '3_168_7', '3_189_7', '3_81_1', '3_211_6', '3_109_9', '3_218_3', '5_211_9', '3_115_4', '3_221_2', '3_102_1', '3_107_7', '3_216_9', '3_210_4', '3_144_4', '2_214_8', '2_101_9', '1_156_9', '3_194_5', '2_218_3', '3_197_5', '5_117_1', '3_77_7', '3_208_3', '2_218_2', '2_198_3', '3_127_7', '3_82_3', '3_174_4', '3_133_6', '2_221_5', '3_85_4', '2_100_6', '2_201_6', '2_197_8', '2_93_9', '2_87_4', '3_196_4', '3_162_1', '2_170_8', '2_178_6', '2_221_6', '3_171_1', '3_66_5', '5_193_3', '1_160_3', '1_65_6', '3_204_5', '5_221_9', '3_214_9', '2_85_3', '5_222_1

# Only export this to a pickle file if you plan to use the results and the mmm_data in another notebook

In [10]:
import pickle

with open("pareto_result.pkl", "wb") as f:
    pickle.dump(pareto_result, f)
with open("mmmdata.pkl", "wb") as f:
    pickle.dump(mmm_data, f)